In [ ]:
import pandas as pd
from geopy.distance import geodesic
from tqdm import tqdm

# Load the CSV file
file_path = r'C:\Users\nicho\Downloads\ENUGU_crosschecked_with_coordinates_corrected1.csv'
df = pd.read_csv(file_path)

# Print column names to check for the exact names
print("Columns in the DataFrame:", df.columns)

# Ensure Latitude and Longitude columns exist and are named correctly
if 'Latitude' not in df.columns or 'Longitude' not in df.columns:
    raise KeyError("The DataFrame does not contain 'Latitude' and 'Longitude' columns. Please check the column names.")

# Convert Latitude and Longitude to numeric
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

# Define the radius in kilometers
radius_km = 1

# Function to calculate distance and identify neighbors
def find_neighbors(df, radius_km):
    neighbors = {}
    # Initialize tqdm for progress tracking
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Identifying neighbors"):
        current_unit = (row['Latitude'], row['Longitude'])
        neighbors[idx] = []
        for i, other_row in df.iterrows():
            if idx != i:
                other_unit = (other_row['Latitude'], other_row['Longitude'])
                if pd.notna(current_unit[0]) and pd.notna(current_unit[1]) and pd.notna(other_unit[0]) and pd.notna(
                        other_unit[1]):
                    distance = geodesic(current_unit, other_unit).km
                    if distance <= radius_km:
                        neighbors[idx].append(i)
    return neighbors

# Identify neighbors
neighbors_dict = find_neighbors(df, radius_km)

# Convert neighbor indices to their corresponding names or IDs
df['Neighbors'] = df.index.map(lambda idx: ', '.join(df.loc[neighbors_dict[idx], 'PU-Name'].astype(str)))

# Save the updated dataframe to an Excel file
output_file_path = r'C:\Users\nicho\Downloads\file_2.xlsx'
df.to_excel(output_file_path, index=False)
print(f"Updated file saved as '{output_file_path}'")